In [59]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import *
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob
from tweets_ops import *
from url_helpers import analyze_url
from matching import *
from textstat.textstat import textstat
import urllib, bs4

pd.options.mode.chained_assignment = None

import warnings
warnings.simplefilter("ignore", category=UserWarning)

figures = conf['aux_dir']+'figure-eight/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
df6 = pd.read_csv(cache_dir+'article_details_v6.tsv', sep='\t')
df7 = pd.read_csv(cache_dir+'article_details_v7.tsv', sep='\t').fillna(0).drop('Alexa Rank', axis=1)
df7['Author Signature'] = df6['authors'].apply(lambda x: len(eval(x)) != 0)


In [61]:
df_crispr = pd.read_csv(cache_dir+'crispr.tsv', sep='\t').merge(df7, left_on='article', right_on='url')
dfD = pd.read_csv(figures+'crispr-Dimitra.csv').rename(columns={'how_do_you_rate_the_scientific_quality_of_this_article': 'expert1'}).sort_values(by='article')
dfJ = pd.read_csv(figures+'crispr-Jose.csv').rename(columns={'how_do_you_rate_the_scientific_quality_of_this_article': 'expert2'}).sort_values(by='article').drop('article', axis=1)
df_exp_crispr = pd.concat([dfD,dfJ], axis=1)[['article', 'expert1', 'expert2']]
df_exp_crispr.article = df_exp_crispr.article.apply(lambda x: x.replace('https://', 'http://'))
df_exp_crispr['expert'] = (df_exp_crispr['expert1'] + df_exp_crispr['expert2'])/2
df_crispr = df_crispr.merge(df_exp_crispr[['article', 'expert']])

df_atc = pd.read_csv(cache_dir+'alc_tob_caf.tsv', sep='\t').merge(df7, left_on='article', right_on='url')
dfAn = pd.read_csv(figures+'tobacco-Andreu.csv').rename(columns={'how_do_you_rate_the_scientific_quality_of_this_article': 'expert1'}).sort_values(by='article')
dfAi = pd.read_csv(figures+'tobacco-Aina.csv').rename(columns={'how_do_you_rate_the_scientific_quality_of_this_article': 'expert2'}).sort_values(by='article').drop('article', axis=1)
df_exp_atc = pd.concat([dfAn,dfAi], axis=1)[['article', 'expert1', 'expert2']]
df_exp_atc.article = df_exp_atc.article.apply(lambda x: x.replace('https://', 'http://'))
df_exp_atc['expert'] = (df_exp_atc['expert1'] + df_exp_atc['expert2'])/2
df_atc = df_atc.merge(df_exp_atc[['article', 'expert']])

In [62]:
X_crispr = np.array(df_crispr.drop(['article', 'paper', 'title', 'url', 'expert'], axis=1).values, dtype=np.float32)
y_crispr = df_crispr['expert'].values
y_crispr = y_crispr.round().astype(int)

X_atc = np.array(df_atc.drop(['article','paper1', 'title1', 'paper2', 'title2', 'paper3', 'title3', 'title', 'url', 'expert'], axis=1).values, dtype=np.float32)
y_atc = df_atc['expert'].values
y_atc = y_atc.round().astype(int)

In [63]:
df7['url'] = df7['url'].apply(lambda x: analyze_url(x)[0])
df7 = df7.merge(pd.read_csv(cache_dir+'acsh.tsv', sep='\t'), left_on='url', right_on='outlet')

In [85]:
df8=df7
#df8 = df7[df7['url'].isin(['nytimes.com', 'theatlantic.com', 'dailymail.co.uk'])] #theatlantic.com, dailymail.co.uk, nytimes.com

In [86]:
X_train = np.array(df8.drop(['outlet', 'url', 'rate'], axis=1).values, dtype=np.float32)
y_train = df8['rate'].values

In [88]:
n_est = 250
m_dep = 200

classifier = RandomForestClassifier(n_estimators=n_est, max_depth=m_dep, n_jobs=-1, random_state=42)

classifier.fit(X_train, y_train)
imp = classifier.feature_importances_

d = {c:r for c, r in zip(df7.drop(['outlet', 'url', 'rate'], axis=1).columns, imp)}
pd.DataFrame(sorted(d.items(), key=lambda kv: kv[1], reverse=True))[0][:6]


# classifier.fit(X_train, y_train)
# print(n_est, m_dep, classifier.score(X_atc, y_atc))
# df_atc['scilens'] = classifier.predict(X_atc)

#classifier.fit(X_train, y_train)
#print(n_est, m_dep, classifier.score(X_crispr, y_crispr))    
# df_crispr['scilens'] = classifier.predict(X_crispr)

0    Article Word Count
1           Readability
2        #Users Friends
3            #Followers
4         #Users Tweets
5                   STS
Name: 0, dtype: object

In [43]:
df_atc[['article', 'scilens']].to_csv(cache_dir+'scilens_atc.tsv', sep='\t', index=None)